In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo

In [2]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [79]:
 def scrape():
    scrapeData = {}
    
    url = "https://mars.nasa.gov/news/?page=0&per_page=15&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

    browser.visit(url)

    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')

    content = soup.find('div', class_='article_teaser_body').get_text()

    title = soup.find('div', class_='content_title').get_text()

    scrapeData["latest_news_title"] = title
    
    scrapeData["latest_news_content"] = content

    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

    browser.visit(url)

    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')

    feature_image = "https://www.jpl.nasa.gov" + soup.find('a', class_='button fancybox')['data-fancybox-href']
    
    scrapeData["feature_image"] = feature_image
    
    url = "https://twitter.com/marswxreport?lang=en"

    browser.visit(url)

    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')

    mars_weather = soup.find(class_='js-tweet-text-container').find(class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").get_text()
    
    scrapeData["mars_weather"] = mars_weather
    
    url = "https://space-facts.com/mars/"

    browser.visit(url)

    html = browser.html

    soup = BeautifulSoup(html, 'html.parser')

    table = soup.find(id="tablepress-mars")

    import pandas as pd 

    df = pd.read_html(str(table))

    df = df[0]

    mars_facts = pd.Series(df[1].values, index = df[0]).to_dict() 
    
    scrapeData["mars_facts"] = mars_facts
    
    url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

    browser.visit(url)

    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')

    mars_links = []

    img_links = soup.select("div.description")

    for i in img_links:
        url = "https://astrogeology.usgs.gov" + i.find("a",class_="itemLink")["href"]
        mars_links.append(url)

    mars_images = {}

    for link in mars_links:
        browser.visit(link)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        img_url = "https://astrogeology.usgs.gov" + soup.find(class_="wide-image")["src"]
        hemisphere_title = browser.title.split("|")[0][:-1]
        mars_images[hemisphere_title] = img_url 
        
    scrapeData["mars_images"] = mars_images
    
    return scrapeData
    

In [80]:
scrape()

{'feature_image': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18289_ip.jpg',
 'latest_news_content': "A recent view from Mars orbit of the site where NASA's Phoenix Mars mission landed on far-northern Mars nearly a decade ago captures changes.",
 'latest_news_title': 'Nearly a Decade After Mars Phoenix Landed, Another Look',
 'mars_facts': {'Equatorial Diameter:': '6,792 km',
  'First Record:': '2nd millennium BC',
  'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.52 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Polar Diameter:': '6,752 km',
  'Recorded By:': 'Egyptian astronomers',
  'Surface Temperature:': '-153 to 20 °C'},
 'mars_images': {'Cerberus Hemisphere Enhanced': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'Schiaparelli Hemisphere Enhanced': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_sc